Capstone Project - The Battle of Neighborhoods (Week 1-2)

Business Problem
The business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

Data section
Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

 Methodology section
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

1. Collect Inspection Data
After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [2]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [13]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv")

2. Explore and Understand Data
We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [14]:
df_ppd.head(5)

,{A2479555-84B1-74C7-E053-6B04A8C0887D},266000,2020-02-13 00:00,YO26 5TB,T,N,F,1,Unnamed: 8,MONARCH WAY,Unnamed: 10,YORK,YORK.1,YORK.2,A,A.1
0,{A2479555-84B2-74C7-E053-6B04A8C0887D},315000,2020-02-14 00:00,LS25 6AX,D,N,F,67,NaN,CHURCH HILL,SHERBURN IN ELMET,LEEDS,SELBY,NORTH YORKSHIRE,A,A
1,{A2479555-84B3-74C7-E053-6B04A8C0887D},136000,2020-03-06 00:00,LS24 9GZ,F,N,L,28,NaN,BRIDGE CLOSE,CHURCH FENTON,TADCASTER,SELBY,NORTH YORKSHIRE,A,A
2,{A2479555-84B4-74C7-E053-6B04A8C0887D},155000,2020-02-28 00:00,YO7 1HW,S,N,F,4,NaN,GEORGE COURT,SOWERBY,THIRSK,HAMBLETON,NORTH YORKSHIRE,A,A
3,{A2479555-84B5-74C7-E053-6B04A8C0887D},261000,2020-02-17 00:00,YO7 1SB,S,N,F,30,NaN,SAXTY WAY,SOWERBY,THIRSK,HAMBLETON,NORTH YORKSHIRE,A,A
4,{A2479555-84B6-74C7-E053-6B04A8C0887D},405000,2020-02-21 00:00,YO26 6NS,D,N,F,29,NaN,EASTHORPE DRIVE,NETHER POPPLETON,YORK,YORK,YORK,A,A


In [15]:
df_ppd.shape

(112459, 16)

3. Data preparation and preprocessing
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

Rename the column names
Format the date column
Sort data by date of sale
Select data only for the city of London
Make a list of street names in London
Calculate the street-wise average price of the property
Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
Join the data to find the coordinates of locations which fit into client's budget
Plot recommended locations on London map along with current market prices

In [16]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [17]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)


# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [18]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [19]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [20]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [21]:
# Display the dataframe
df_affordable

,Street,Avg_Price
48,AIREDALE AVENUE,2.300000e+06
69,ALDERNEY STREET,2.260000e+06
273,BASILICA MEWS,2.200000e+06
397,BLAKE GARDENS,2.500000e+06
636,CADOGAN SQUARE,2.250000e+06
729,CASTELNAU,2.370000e+06
804,CHELSEA EMBANKMENT,2.500000e+06
861,CHOLMELEY CRESCENT,2.350000e+06
866,CHRISTINA STREET,2.350000e+06
901,CLARENDON ROAD,2.279500e+06


In [22]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [23]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 48
item: Street       AIREDALE AVENUE
Avg_Price            2.3e+06
Name: 48, dtype: object
item.Street only: AIREDALE AVENUE
index: 69
item: Street       ALDERNEY STREET
Avg_Price           2.26e+06
Name: 69, dtype: object
item.Street only: ALDERNEY STREET
index: 273
item: Street       BASILICA MEWS
Avg_Price          2.2e+06
Name: 273, dtype: object
item.Street only: BASILICA MEWS
index: 397
item: Street       BLAKE GARDENS
Avg_Price          2.5e+06
Name: 397, dtype: object
item.Street only: BLAKE GARDENS
index: 636
item: Street       CADOGAN SQUARE
Avg_Price          2.25e+06
Name: 636, dtype: object
item.Street only: CADOGAN SQUARE
index: 729
item: Street       CASTELNAU
Avg_Price     2.37e+06
Name: 729, dtype: object
item.Street only: CASTELNAU
index: 804
item: Street       CHELSEA EMBANKMENT
Avg_Price               2.5e+06
Name: 804, dtype: object
item.Street only: CHELSEA EMBANKMENT
index: 861
item: Street       CHOLMELEY CRESCENT
Avg_Price              2.35e+06
Name: 861

In [24]:
geolocator = Nominatim()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [25]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
df_affordable

,Street,Avg_Price,city_coord
48,AIREDALE AVENUE,2.300000e+06,"(53.4363678, -1.106305)"
69,ALDERNEY STREET,2.260000e+06,"(52.9458857, -1.170078)"
273,BASILICA MEWS,2.200000e+06,"(40.74874174999999, 14.484493702349866)"
397,BLAKE GARDENS,2.500000e+06,"(22.2842798, 114.14818268316824)"
636,CADOGAN SQUARE,2.250000e+06,"(51.4943741, -0.1609482)"
729,CASTELNAU,2.370000e+06,"(43.58041, -0.02933)"
804,CHELSEA EMBANKMENT,2.500000e+06,"(51.4843415, -0.1593009)"
861,CHOLMELEY CRESCENT,2.350000e+06,"(51.5723967, -0.1451051)"
866,CHRISTINA STREET,2.350000e+06,"(35.053383, -79.068335)"
901,CLARENDON ROAD,2.279500e+06,"(51.5060693, -3.1544318)"


In [27]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [28]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
48,AIREDALE AVENUE,2.300000e+06,"(53.4363678, -1.106305)",53.436368,-1.106305
69,ALDERNEY STREET,2.260000e+06,"(52.9458857, -1.170078)",52.945886,-1.170078
273,BASILICA MEWS,2.200000e+06,"(40.74874174999999, 14.484493702349866)",40.748742,14.484494
397,BLAKE GARDENS,2.500000e+06,"(22.2842798, 114.14818268316824)",22.284280,114.148183
636,CADOGAN SQUARE,2.250000e+06,"(51.4943741, -0.1609482)",51.494374,-0.160948
729,CASTELNAU,2.370000e+06,"(43.58041, -0.02933)",43.580410,-0.029330
804,CHELSEA EMBANKMENT,2.500000e+06,"(51.4843415, -0.1593009)",51.484341,-0.159301
861,CHOLMELEY CRESCENT,2.350000e+06,"(51.5723967, -0.1451051)",51.572397,-0.145105
866,CHRISTINA STREET,2.350000e+06,"(35.053383, -79.068335)",35.053383,-79.068335
901,CLARENDON ROAD,2.279500e+06,"(51.5060693, -3.1544318)",51.506069,-3.154432


In [29]:
df = df_affordable.drop(columns=['city_coord'])

In [30]:
df

,Street,Avg_Price,Latitude,Longitude
48,AIREDALE AVENUE,2.300000e+06,53.436368,-1.106305
69,ALDERNEY STREET,2.260000e+06,52.945886,-1.170078
273,BASILICA MEWS,2.200000e+06,40.748742,14.484494
397,BLAKE GARDENS,2.500000e+06,22.284280,114.148183
636,CADOGAN SQUARE,2.250000e+06,51.494374,-0.160948
729,CASTELNAU,2.370000e+06,43.580410,-0.029330
804,CHELSEA EMBANKMENT,2.500000e+06,51.484341,-0.159301
861,CHOLMELEY CRESCENT,2.350000e+06,51.572397,-0.145105
866,CHRISTINA STREET,2.350000e+06,35.053383,-79.068335
901,CLARENDON ROAD,2.279500e+06,51.506069,-3.154432


In [31]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [32]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [33]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'FNIOGD3FYWSMQMZ0ZSK4552NX4MK4NHQN4WKQW155002GAQV' # Foursquare ID
CLIENT_SECRET = 'GOJI0LHSJHUKV4PWX3CURDAKHCUH5TISUCFTT0SJURB01WZC' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FNIOGD3FYWSMQMZ0ZSK4552NX4MK4NHQN4WKQW155002GAQV
CLIENT_SECRET:GOJI0LHSJHUKV4PWX3CURDAKHCUH5TISUCFTT0SJURB01WZC


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

4. Modeling
After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

AIREDALE AVENUE
ALDERNEY STREET
BASILICA MEWS
BLAKE GARDENS
CADOGAN SQUARE
CASTELNAU
CHELSEA EMBANKMENT
CHOLMELEY CRESCENT
CHRISTINA STREET
CLARENDON ROAD
CLAREVILLE STREET
COLEHERNE MEWS
CRISTOWE ROAD
CROCKERTON ROAD
CRONDACE ROAD
DUNDONALD ROAD
EAST DULWICH ROAD
GEORGE STREET
HARTINGTON ROAD
HESTER ROAD
HOXTON SQUARE
KENSINGTON GORE
LILYVILLE ROAD
LOGAN PLACE
LOWNDES SQUARE
MITCHAM ROAD
NETHERHALL GARDENS
OLD BREWERY MEWS
OPPIDANS ROAD
ORMISTON GROVE
POND STREET
PRIORY TERRACE
RADIPOLE ROAD
RIVER STREET MEWS
ROSEBERY AVENUE
SANDY ROAD
STRAND
TEMPLE SHEEN
TUFNELL PARK ROAD
WELLGARTH ROAD
WESTBOURNE GROVE
WILMINGTON SQUARE


In [36]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIREDALE AVENUE,53.436368,-1.106305,Carpenters arms,53.436829,-1.103640,Pub
1,AIREDALE AVENUE,53.436368,-1.106305,Heating Yorkshire,53.438136,-1.101710,Construction & Landscaping
2,AIREDALE AVENUE,53.436368,-1.106305,Scarbrough Arms,53.432736,-1.106993,Pub
3,AIREDALE AVENUE,53.436368,-1.106305,Holmes & Co Bridal Couture,53.432335,-1.109214,Bridal Shop
4,AIREDALE AVENUE,53.436368,-1.106305,Rocco's Italian Kitchen,53.432410,-1.109580,Italian Restaurant
...,...,...,...,...,...,...,...
1261,WILMINGTON SQUARE,51.526765,-0.110485,The Apple Tree,51.523638,-0.112068,Pub
1262,WILMINGTON SQUARE,51.526765,-0.110485,Luce e Limoni,51.523332,-0.114662,Italian Restaurant
1263,WILMINGTON SQUARE,51.526765,-0.110485,Calthorpe Arms,51.524837,-0.116328,Pub
1264,WILMINGTON SQUARE,51.526765,-0.110485,Sekforde,51.524352,-0.104511,Pub


In [37]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
AIREDALE AVENUE,5,5,5,5,5,5
ALDERNEY STREET,13,13,13,13,13,13
BASILICA MEWS,38,38,38,38,38,38
BLAKE GARDENS,66,66,66,66,66,66
CADOGAN SQUARE,100,100,100,100,100,100
CHELSEA EMBANKMENT,10,10,10,10,10,10
CHOLMELEY CRESCENT,31,31,31,31,31,31
CHRISTINA STREET,1,1,1,1,1,1
CLARENDON ROAD,3,3,3,3,3,3


In [38]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 220 uniques categories.


In [39]:
location_venues.shape

(1266, 7)

In [40]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Adult Boutique,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,University,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Adult Boutique,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,University,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,AIREDALE AVENUE,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,ALDERNEY STREET,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,BASILICA MEWS,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,BLAKE GARDENS,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.0,...,0.000000,0.000000,0.015152,0.015152,0.000000,0.015152,0.015152,0.000000,0.000000,0.0
4,CADOGAN SQUARE,0.00,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
5,CHELSEA EMBANKMENT,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1
6,CHOLMELEY CRESCENT,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,CHRISTINA STREET,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,CLARENDON ROAD,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,CLAREVILLE STREET,0.00,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.0,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000,0.0


In [42]:
london_grouped.shape

(38, 221)

In [43]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AIREDALE AVENUE----
                        venue  freq
0                         Pub   0.4
1                 Bridal Shop   0.2
2  Construction & Landscaping   0.2
3          Italian Restaurant   0.2
4               Jewelry Store   0.0


----ALDERNEY STREET----
                 venue  freq
0                  Pub  0.15
1                Hotel  0.15
2  Japanese Restaurant  0.08
3                  Bar  0.08
4                 Café  0.08


----BASILICA MEWS----
                venue  freq
0       Historic Site  0.24
1      History Museum  0.18
2  Italian Restaurant  0.16
3         Pizza Place  0.08
4          Campground  0.05


----BLAKE GARDENS----
                 venue  freq
0  Japanese Restaurant  0.11
1                 Café  0.09
2     Tapas Restaurant  0.05
3          Coffee Shop  0.05
4    French Restaurant  0.05


----CADOGAN SQUARE----
                venue  freq
0               Hotel  0.10
1                Café  0.06
2          Restaurant  0.06
3            Boutique  0.06
4  It

In [44]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [46]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [47]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AIREDALE AVENUE,Pub,Bridal Shop,Italian Restaurant,Construction & Landscaping,English Restaurant,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
1,ALDERNEY STREET,Hotel,Pub,Coffee Shop,Grocery Store,Optical Shop,Café,Bar,Gym,Fast Food Restaurant,Japanese Restaurant
2,BASILICA MEWS,Historic Site,History Museum,Italian Restaurant,Pizza Place,Campground,Diner,Hotel,Train Station,National Park,Supermarket
3,BLAKE GARDENS,Japanese Restaurant,Café,Tapas Restaurant,French Restaurant,Coffee Shop,Thai Restaurant,Bar,Restaurant,Chinese Restaurant,Cocktail Bar
4,CADOGAN SQUARE,Hotel,Café,Boutique,Italian Restaurant,Restaurant,Clothing Store,Indian Restaurant,Pub,Gym / Fitness Center,Ice Cream Shop


In [48]:
venues_sorted.shape

(38, 11)

In [49]:
london_grouped.shape

(38, 221)

In [50]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [51]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([0, 2, 2, 3, 2, 4, 3, 4, 4, 0, 2, 4, 3, 1, 0, 1, 2, 4, 2, 1, 0, 3,
       2, 2, 1, 4, 2, 0, 2, 2, 4, 0, 2, 1, 1, 0, 0, 2, 4, 3, 0, 1],
      dtype=int32)

In [52]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
48,AIREDALE AVENUE,2300000.0,53.436368,-1.106305
69,ALDERNEY STREET,2260000.0,52.945886,-1.170078
273,BASILICA MEWS,2200000.0,40.748742,14.484494
397,BLAKE GARDENS,2500000.0,22.284280,114.148183
636,CADOGAN SQUARE,2250000.0,51.494374,-0.160948


In [53]:
london_grouped_clustering.shape

(42, 4)

In [54]:
df.shape

(42, 4)

In [55]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [56]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [57]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,AIREDALE AVENUE,2.300000e+06,53.436368,-1.106305,0,Pub,Bridal Shop,Italian Restaurant,Construction & Landscaping,English Restaurant,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
69,ALDERNEY STREET,2.260000e+06,52.945886,-1.170078,2,Hotel,Pub,Coffee Shop,Grocery Store,Optical Shop,Café,Bar,Gym,Fast Food Restaurant,Japanese Restaurant
273,BASILICA MEWS,2.200000e+06,40.748742,14.484494,2,Historic Site,History Museum,Italian Restaurant,Pizza Place,Campground,Diner,Hotel,Train Station,National Park,Supermarket
397,BLAKE GARDENS,2.500000e+06,22.284280,114.148183,3,Japanese Restaurant,Café,Tapas Restaurant,French Restaurant,Coffee Shop,Thai Restaurant,Bar,Restaurant,Chinese Restaurant,Cocktail Bar
636,CADOGAN SQUARE,2.250000e+06,51.494374,-0.160948,2,Hotel,Café,Boutique,Italian Restaurant,Restaurant,Clothing Store,Indian Restaurant,Pub,Gym / Fitness Center,Ice Cream Shop
729,CASTELNAU,2.370000e+06,43.580410,-0.029330,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
804,CHELSEA EMBANKMENT,2.500000e+06,51.484341,-0.159301,3,Café,Garden,Zoo,Harbor / Marina,French Restaurant,Monument / Landmark,Plaza,Pub,Asian Restaurant,Ethiopian Restaurant
861,CHOLMELEY CRESCENT,2.350000e+06,51.572397,-0.145105,4,Pub,Café,Indian Restaurant,Bakery,Tea Room,Seafood Restaurant,Coffee Shop,Plaza,Pizza Place,Deli / Bodega
866,CHRISTINA STREET,2.350000e+06,35.053383,-79.068335,4,Grocery Store,Zoo,Electronics Store,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Factory
901,CLARENDON ROAD,2.279500e+06,51.506069,-3.154432,0,Playground,Gym / Fitness Center,Gym,Zoo,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Factory


In [58]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,2300000.0,Pub,Bridal Shop,Italian Restaurant,Construction & Landscaping,English Restaurant,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
901,2279500.0,Playground,Gym / Fitness Center,Gym,Zoo,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Factory
1086,2275000.0,Coffee Shop,Grocery Store,Café,Pub,French Restaurant,Mediterranean Restaurant,Climbing Gym,Japanese Restaurant,Park,Cycle Studio
2138,2320000.0,Coffee Shop,Café,Hotel,Cocktail Bar,Art Gallery,Italian Restaurant,Pub,Bar,Beer Bar,Restaurant
2982,2300000.0,Café,Pub,Bakery,Ice Cream Shop,Japanese Restaurant,Italian Restaurant,Pizza Place,Museum,Coffee Shop,Monument / Landmark


In [60]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1073,2.403333e+06,Indian Restaurant,Pub,Bakery,Café,Spa,Middle Eastern Restaurant,Burger Joint,Metro Station,Gas Station,Pakistani Restaurant
1275,2.450000e+06,Pool,Pizza Place,Park,Train Station,Grocery Store,Gym,Café,Pub,Cupcake Shop,Dog Run
2020,2.452500e+06,Bar,Food,Convenience Store,BBQ Joint,English Restaurant,Food & Drink Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
2508,2.430000e+06,Hotel,Boutique,Café,Jewelry Store,Grocery Store,Restaurant,Plaza,Department Store,Lounge,Shoe Store
3420,2.450000e+06,Historic Site,Middle Eastern Restaurant,Hostel,Pizza Place,Falafel Restaurant,History Museum,Hotel,Coffee Shop,Mediterranean Restaurant,Café


In [61]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,2260000.0,Hotel,Pub,Coffee Shop,Grocery Store,Optical Shop,Café,Bar,Gym,Fast Food Restaurant,Japanese Restaurant
273,2200000.0,Historic Site,History Museum,Italian Restaurant,Pizza Place,Campground,Diner,Hotel,Train Station,National Park,Supermarket
636,2250000.0,Hotel,Café,Boutique,Italian Restaurant,Restaurant,Clothing Store,Indian Restaurant,Pub,Gym / Fitness Center,Ice Cream Shop
902,2250000.0,Hotel,Bakery,Italian Restaurant,Café,Sandwich Place,Garden,Burger Joint,Ice Cream Shop,Tapas Restaurant,Indian Restaurant
1311,2250000.0,Pub,Coffee Shop,Café,Cocktail Bar,Park,Restaurant,Pizza Place,Burger Joint,Chinese Restaurant,Mexican Restaurant


In [62]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
397,2500000.0,Japanese Restaurant,Café,Tapas Restaurant,French Restaurant,Coffee Shop,Thai Restaurant,Bar,Restaurant,Chinese Restaurant,Cocktail Bar
804,2500000.0,Café,Garden,Zoo,Harbor / Marina,French Restaurant,Monument / Landmark,Plaza,Pub,Asian Restaurant,Ethiopian Restaurant
1072,2485000.0,Pub,Café,Italian Restaurant,Park,Grocery Store,Coffee Shop,Yoga Studio,Juice Bar,Bakery,Indian Restaurant
2244,2475000.0,Café,Science Museum,Garden,Gym / Fitness Center,Bar,Monument / Landmark,Student Center,Bookstore,Fountain,Champagne Bar
4285,2500000.0,Park,Bus Stop,Fast Food Restaurant,Mattress Store,Food,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Falafel Restaurant,Factory


In [63]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
729,2370000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,2350000.0,Pub,Café,Indian Restaurant,Bakery,Tea Room,Seafood Restaurant,Coffee Shop,Plaza,Pizza Place,Deli / Bodega
866,2350000.0,Grocery Store,Zoo,Electronics Store,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Factory
950,2350000.0,Italian Restaurant,Pub,Hotel,Pizza Place,Garden,Café,Speakeasy,Bar,Park,Farmers Market
1667,2382000.0,Café,Hotel,Bar,Art Gallery,Cocktail Bar,Coffee Shop,Park,Breakfast Spot,Pub,Indian Restaurant


Results and Discussion section
First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

Conclusion
To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.